##A Sock Drawer

Let us model a sock drawer.  How does a sock drawer work?  Well, when you do the washing, you end up with a bunch of socks, and you just put them in the drawer. So one operation is "put sock in drawer".  The other thing you typically do is look for two matching socks.  For simplicity, we will have only one attribute that we use to match socks: their color.  So the operations, in detail, are:

* `add_sock`: adds a sock of a given color
* `get_pair`: ask if there is a pair of a given color, and if so, remove it
* `available_colors`: returns the set of colors for which there is at least a pair.
  



Here is how we plan to use the sock drawer:

    # Creates a sock drawer and puts some socks in it.
    sd = SockDrawer()
    sd.add_sock("red")
    sd.add_sock("green")
    sd.add_sock("red")
    
    # Gets the available colors
    colors = sd.available_colors
    
    # Picks a possible color
    import random
    color = random.choice(list(colors))

    # And a sock of that color.
    sock = sd.get_pair(color)
    print("gotten sock:", sock, "of color:", color)

An aside: why does `available_colors` return a set and not a list?
For two reasons.  First, returning a set makes clear (and indeed, enforces) that every color should appear only once in the return result.
Second, in sets, the order of elements is irrelevant.  If we used a list, rather than a set, in any unit test suite we would have to:

* Check that every element appears only once in the list;
* sort the list of results before comparing it with the expected results, to ensure that different orders of elements in the returned list do not cause the test to fail.

In general, it is quite important to choose as types for variables or return values, a type that expresses as best as possible the intended properties of the value.

Another thing: we want to define `available_colors` so that it is a _property_ rather than a method.  In this way, for a sock drawer `d`, we will be able to write `d.available_colors` rather than `d.available_colors()`.  It is a small difference, but in my opinion it does make the code look a little bit cleaner.

In [ ]:
class SockDrawer(object):

    def __init__(self):
        self.drawer = []

    def add_sock(self, color):
        """Adds a sock of the given color to the drawer."""
        self.drawer.append(color)

    def get_pair(self, color):
        """Returns False if there is no pair of the given color, and True
        if there is.  In the latter case, it removes the pair from the drawer."""
        if self.drawer.count(color) >= 2:
            self.drawer.remove(color)
            self.drawer.remove(color)
            return True
        else:
            return False

    @property
    def available_colors(self):
        """Lists the colors for which we have at least two socks available."""
        colors = set()
        for el in self.drawer:
            if self.drawer.count(el) >= 2:
                colors.add(el)
        return colors


Let us give this a spin, with our intended use:

In [ ]:
# Creates a sock drawer and puts some socks in it.
sd = SockDrawer()
sd.add_sock("red")
sd.add_sock("green")
sd.add_sock("red")

# Gets the available colors
colors = sd.available_colors

# Picks a possible color
import random
color = random.choice(list(colors))

# And a sock of that color.
sock = sd.get_pair(color)
print("gotten sock:", sock, "of color:", color)


gotten sock: True of color: red


There shouldn't be any more sock pairs available:

In [ ]:
sd.available_colors


set()

In the above implementation, `@property` is a [_decorator_](https://wiki.python.org/moin/PythonDecorators).  From the practical point of view, the [`@property` decorator](https://docs.python.org/3/library/functions.html#property) enables us to access the available colors of a sock drawer `sd` via `sd.available_colors` rather than `sd.available_colors()`: that is, it makes what is in truth a method call look like a variable access.  Big deal, you might say; all it does is it removes the need for parentheses!  True, but the eye also wants its part.

In [ ]:
#Let's define a testing helper.

def check_equal(x, y, msg=None):
    if x != y:
        if msg is None:
            print("Error:")
        else:
            print("Error in", msg, ":")
        print("    Your answer was:", x)
        print("    Correct answer: ", y)
    assert x == y, "%r and %r are different" % (x, y)


In [ ]:
def test_drawer(c):
    """Tests a drawer class c."""
    d = c()
    d.add_sock('red')
    d.add_sock('red')
    d.add_sock('red')
    d.add_sock('green')
    check_equal(d.available_colors, {'red'})
    check_equal(d.get_pair('red'), True)
    check_equal(d.get_pair('green'), False)
    check_equal(d.get_pair('red'), False)
    d.add_sock('blue')
    d.add_sock('blue')
    d.add_sock('blue')
    d.add_sock('red')
    check_equal(d.available_colors, {'red', 'blue'})


In [ ]:
test_drawer(SockDrawer)


I always find it a bit anticlimatic when all the tests pass, and all you get for this accomplishment is... nothing.  Yes, we could define a `PatOnTheBack` exception and raise it, but that will be for another time.

Rather, let us discuss why the above implementation of a sock drawer is not great.
A list keeps elements in a ... list.  So if you need to count the number of times in which an element occurs, you need to traverse the list.  The method count() is faster than the do-it-yourself solution `len([x for x in self.drawer if x == color])`,  as it is implemented natively, but it is still slow.  On top of that, once having counted the number of colors, if the chosen color appears more than once in `get_pair`, we call remove() _twice_ to remove it.  This is just esthetically terrible. We go over the list three times, each time unable to make use of the information we found the previous time.

In the big scheme of things, the implementation might be fine.  "Going over the list" takes time proportional to the length of the list (it is a _linear-time_ operation), and unless a person has a very large number of socks, the performance may never become a bottleneck.  What is really disturbing, to me at least, is the ugliness of the code.

## A Better Sock Drawer

Let us aim at a better solution.  The problem with our list-based implementation, fundamentally, is that all our questions can be answered by knowing how many socks we have for each color, and the list implementation stores this information in unary format, and mixing all colors together.  A much more rational way of remembering the status of the drawer is to associate with each color the number of socks we have.  Thus, our main data structure will be a dictionary mapping colors to integers.

A short aside. If `d` is a normal dictionary, and you want to add `quantity` to `color`, you end up writing code like this:

In [ ]:
# Just to initialize
d = {'green': 4}
color = 'red'
quantity = 2

if color in d:
    d[color] += quantity
else:
    d[color] = quantity


Yes, you can do a little better, as in:

In [ ]:
d[color] = d.get(color, 0) + quantity


but this is not nearly as elegant as being able to access a dictionary with square-bracket notation.  We would like some way of avoiding the test, or the concern, of whether a key is already in the dictionary.  



### defaultdicts

The [`defaultdict` collection type](https://docs.python.org/3/library/collections.html#collections.defaultdict) does precisely this.  A defaultdict dictionary, when accessed, returns the value associated with the key, if any, and otherwise returns a default value.  You specify the default value when you create the defaultdict, by passing to its creator a _factory_ function.  The factory function is called any time a default value is needed, and should return the default value.

In our case, we want a dictionary where the default value is 0; so we use the function `int` which, when called without arguments, returns 0.  Yes, it is a little confusing that `int` is both a type and a function...

In [ ]:
int()


0

So we can create a dictionary with default value 0 as follows:

In [ ]:
from collections import defaultdict
d = defaultdict(int)


In [ ]:
d


defaultdict(int, {})

When an element is not found, rather than throwing an error, our defaultdict returns value 0:

In [ ]:
d['red']


0

And we can write code like this:

In [ ]:
d['red'] += 2
print(d['red'])
print(d['green'])


2
0


With this we can define a better sock drawer:

In [ ]:
class SmartDrawer(object):

    def __init__(self):
        self.socks = defaultdict(int)

    def add_sock(self, color):
        self.socks[color] += 1

    def get_pair(self, color):
        if self.socks[color] > 1:
            self.socks[color] -= 2
            return True
        else:
            return False

    @property
    def available_colors(self):
        return {c for c, n in self.socks.items() if n > 1}


Note how this code is not only more efficient than our previous attempt, but also more concise.  Let us test this class.

In [ ]:
test_drawer(SmartDrawer)


# Arithmetic Dictionaries

At the core of our sock drawer implementation is a fundamental data structure which, for some reason, does not come standard in programming languages: a dictionary that has numerical values (and arbitrary keys).

A number is a number.  An _arithmetic dictionary_ $d$ is a mapping from a set of keys $K$ to numbers (let's say, to $\mathbf{R}$), that is,

$$
d: K \mapsto \mathbf{R} \, .
$$

In other words, $d$ is a function from $K$ to $\mathbf{R}$.

Functions can be combined with arithmetic operators: if you have, for instance, $f_1(x) = x + 2$, and $f_2(x) = x^2$, you can compute $g = f_1 + f_2$, where $g(x) = x^2 + x + 2$.

The same we can do with arithmetic dictionaries: if we have

$$
d_1: K \mapsto \mathbf{R} \qquad d_2: K \mapsto \mathbf{R}
$$

we can add them as $d = d_1 + d_2$, where $d[k] = d_1[k] + d_2[k]$ for all $k \in K$, and similarly for $-$, $/$, and $*$.

In fact, it will be convenient to combine also dictionaries that have different sets of keys.  In that case, if a key $k$ is missing from a dictionary $d$, we will take for $d[k]$ a default value that corresponds to the [_neutral elements_](https://en.wikipedia.org/wiki/Identity_element) with respect to the operation used to combine dictionaries: 0 for $+$ and $-$, and 1 for $\times$ and /.

For example, if we have:

    d1 = {'cat': 4, 'bird': 2}
    d2 = {'cat': 5, 'dog': 3}

and we compute `d = d1 + d2`, we will obtain

    d = {'cat': 9, 'bird': 2, 'dog': 3}

since the missing elements, such as `'bird'` in `d2`, will be assumed to have value 0.

We note that Python includes, in the `collections` module, the [`Counter` class](https://docs.python.org/3.7/library/collections.html#collections.Counter), which supports addition.  Arithmetic dictionaries are a more general version of the `Counter` class.

In [ ]:
class MehAD(dict):

    def __add__(self, other):
        d = MehAD()
        other_keys = set(other.keys()) if isinstance(other, dict) else set()
        for k in set(self.keys()) | other_keys:
            d[k] = self.get(k, 0) + (other.get(k, 0) if isinstance(other, dict) else other)
        return d


In [ ]:
d1 = MehAD()
d2 = MehAD(red=2, green=3) # dict(red=2, green=3)
d1 + d2


{'green': 3, 'red': 2}

In [ ]:
d2 + 3


{'green': 6, 'red': 5}

This would work.  But then, when you set to implement `__sub__`, you would have to repeat more or less the same code, except for `-` instead of `+` in line 10.  And the code for `__mul__` and `__truediv__` would also be very similar.

We want to teach you how to factor the common code.  The idea is to define a method `_binary_op` that takes as arguments the left operand, right operand, the function to be used to combine the operands (addition, subtraction, etc), and the element to be used when the key is not in the dictionary.  All the operands can then be defined in terms of `_binary_op`.  This factorization of common code has two benefits:

* it makes the code more concise, and
* it ensures that, if you find a bug and fix it, _all_ the operations will benefit from the fix.

_Code factorization_ means gathering together the common code, eliminating repetitions; just as factoring $ab + ac$ into $a(b + c)$ removes the repetition of $a$.
It turns out that the deep reason why code factorization is important is not conciseness: rather, it is the fact that once a bug is discovered in the factorized code, all uses of the code will benefit from it.

The code is as follows:

In [ ]:
class AD(dict):

    def __add__(self, other):
        return AD._binary_op(self, other, lambda x, y: x + y, 0)

    def __sub__(self, other):
        return AD._binary_op(self, other, lambda x, y: x - y, 0)

    @staticmethod
    def _binary_op(left, right, op, neutral):
        r = AD()
        l_keys = set(left.keys()) if isinstance(left, dict) else set()
        r_keys = set(right.keys()) if isinstance(right, dict) else set()
        for k in l_keys | r_keys:
            # If the right (or left) element is a dictionary (or an AD),
            # we get the elements from the dictionary; else we use the right
            # or left value itself.  This implements a sort of dictionary
            # broadcasting.
            l_val = left.get(k, neutral) if isinstance(left, dict) else left
            r_val = right.get(k, neutral) if isinstance(right, dict) else right
            r[k] = op(l_val, r_val)
        return r


In the implementation above, we have factored together what would have been the common part of the implementation of $+$ and $-$ in the method `_binary_op`.  The `_binary_op` method is a [_static method:_](https://docs.python.org/3/library/functions.html#staticmethod) it does not refer to a specific object.  In this way, we can decide separately what to provide to it as the left and right hand sides; this will be useful later, as we shall see.  To call it, we have to refer to it via `AD._binary_op` rather than `self._binary_op`, as would be the case for a normal (object) method.  Furthermore, in the `_binary_op` method, we do not have access to `self`, since `self` refers to a specific object, while the method is generic, defined for the class.

The `_binary_op` method combines values from its `left` and `right` operands using operator `op`, and the default `neutral` if a value is not found.  The two operands `left` and `right` are interpreted as dictionaries if they are a subclass of `dict`, and are interpreted as constants otherwise.

We have defined our class AD as a _subclass_ of the dictionary class `dict`.  This enables us to apply to an AD all the [methods that are defined for a dictionary](https://docs.python.org/3/library/stdtypes.html#typesmapping), which is really quite handy.

In [ ]:
d1 = AD()
d1['red'] = 2
print(d1)
d2 = AD({'blue': 4, 'green': 5})
print(d2)


{'red': 2}
{'blue': 4, 'green': 5}


In the code above, note how we can write `d1['red']`: our ability to access ADs via the square-bracket notation is due to the fact that ADs, like dictionaries, implement the [`__getitem__` method](https://docs.python.org/3/reference/datamodel.html#object.__getitem__).  Note also that AD inherits from `dict` even the initializer methods, so that we can pass to our dictionary another dictionary, and obtain an AD that contains a [_copy_ of that dictionary](https://docs.python.org/3/library/stdtypes.html#dict).

We can see that addition and subtraction also work:

In [ ]:
print(AD(red=2, green=3) + AD(red=1, blue=4))
print(AD(red=2, green=3) - AD(red=1, blue=4))


{'red': 3, 'blue': 4, 'green': 3}
{'red': 1, 'blue': -4, 'green': 3}


We let multiplication and division to you to implement.

In [ ]:
### Implement multiplication and division

def ad_mul(self, other):
    return AD._binary_op(self, other, lambda x, y: x * y, 1)


AD.__mul__ = ad_mul

# below division, similarly.
def ad_truediv(self, other):
    return AD._binary_op(self, other, lambda x, y: x / y, 1)

AD.__truediv__ = ad_truediv


# And finally, define below INTEGER division.

def ad_floordiv(self, other):
    return AD._binary_op(self, other, lambda x, y: x // y, 1)

AD.__floordiv__ = ad_floordiv


In [ ]:
### Tests for multiplication

# Note that the elements that are missing in the other dictionary
# are assumed to be 1.
d = AD(a=2, b=3) * AD(b=4, c=5)
check_equal(d, AD(a=2, b=12, c=5))



In [ ]:
### Tests for division

d = AD(a=8, b=6) / AD(a=2, c=4)
check_equal(d, AD(a=4, b=6, c=0.25))



In [ ]:
### Tests for integer division

d = AD(a=8, b=6) // AD(a=2, b=4, c=4)
check_equal(d, AD(a=4, b=1, c=0))



We will let you also implement max_items and min_items properties.  The value of these properties should be a _pair_, consisting of the maximum/minimum value, and of the _set_ of keys that led to that value.  The latter needs to be a set, because there are multiple key, value pairs where the value is maximal or minimal. To avoid wasting your time, we will have you implement only the max_items property.  If the AD is empty, the property's value should be the pair `(None, set())`.

In [ ]:
### Implement `max_items`

def ad_max_items(self):
    """Returns a pair consisting of the max value in the AD, and of the
    set of keys that attain that value. """
    max = 0
    setss = set()
    for k, v in self.items():
      if v > max:
        setss = set()
        setss.add(k)
        max = v
      elif v == max:
        setss.add(k)

    if max == 0:
      return None, set()


    return max, setss

AD.max_items = property(ad_max_items)


In [ ]:
### Tests for `max_items`

# For empty ADs, it has to return None as value, and the empty set as key set.
check_equal(AD().max_items, (None, set()))
# Case of one maximum.
check_equal(AD(red=2, green=3, blue=1).max_items, (3, {'green'}))
# Case of multiple maxima.
check_equal(AD(red=2, yellow=3, blue=3, violet=3, pink=1).max_items,
             (3, {'yellow', 'blue', 'violet'}))


## Comparison and Logical Operators

### Comparisons

If `d` is an `AD`, what is the meaning of `d > 2`?

In numpy, when we have an array `a`, and we write `a > 2`, we obtain the array consisting of boolean entries, indicating whether the elements of `a` are greater than two or not:


In [ ]:
import numpy as np

a = np.array([2, 3, 4, 1, 2])
a > 2


array([False,  True,  True, False, False])

This suggests adopting a similar semantics for inequalities, defining the behavior of the [comparison operators](https://docs.python.org/3/reference/datamodel.html#object.__lt__):

In [ ]:
def ad_lt(self, other):
    return AD._binary_op(self, other, lambda x, y: x < y, 0)

AD.__lt__ = ad_lt


In [ ]:
AD.__lt__ = lambda l, r: AD._binary_op(l, r, lambda x, y: x < y, 0)
AD.__gt__ = lambda l, r: AD._binary_op(l, r, lambda x, y: x > y, 0)
AD.__le__ = lambda l, r: AD._binary_op(l, r, lambda x, y: x <= y, 0)
AD.__ge__ = lambda l, r: AD._binary_op(l, r, lambda x, y: x >= y, 0)


Let us see how this works:

In [ ]:
AD(cat=4, bird=2) > 2


{'bird': False, 'cat': True}

Once we have a dictionary mapping from values to booleans, we need some way of testing whether all keys, or some keys, map to True. In Python, the `all()` function returns whether all elements in a boolean list are true:

In [ ]:
print(all([True, False, True, True]))
print(any([True, True, True, True]))


False
True


So we can define the `all` and `any` methods for an AD as follows:

In [ ]:
def ad_all(self):
    return all(self.values)

AD.all = ad_all


In [ ]:
AD.all = lambda self : all(self.values())
AD.any = lambda self : any(self.values())


We also add a method for filtering the elements that satisfy a condition.  If the condition is left unspecified, we use the truth-value of the elements themselves.

In [ ]:
def ad_filter(self, f=None):
    return AD({k: v for k, v in self.items() if (f(v) if f is not None else v)})
AD.filter = ad_filter


Let us try this out.

In [ ]:
d = AD(green=3, blue=2, red=1)
d.filter(lambda x : x > 1)


{'blue': 2, 'green': 3}

In [ ]:
d = AD(green=3, blue=2, red=1)
(d > 1).filter()


{'blue': True, 'green': True}

In [ ]:
(d > 1).all()


False

In [ ]:
(d > 0).all()


True

## Implementing the Sock Drawer In Terms of Arithmetic Dictionaries

In [ ]:
### The sock drawer class using ADs

class ADSockDrawer(object):

    def __init__(self):
        self.drawer = AD()

    def add_sock(self, color):
        """Adds a sock of the given color to the drawer."""
        self.drawer[color] = self.drawer[color] + 1 if color in self.drawer else 1

    def get_pair(self, color):
        """Returns False if there is no pair of the given color, and True
        if there is.  In the latter case, it removes the pair from the drawer."""
        if self.drawer[color] > 1:
          self.drawer[color] -= 2
          return True
        else:
          return False

    @property
    def available_colors(self):
        """Lists the colors for which we have at least two socks available."""
        return {c for c, n in self.drawer.items() if n > 1}





In [ ]:
# Tests for Sock Drawer implementation based on ADs

test_drawer(ADSockDrawer)

